<a href="https://colab.research.google.com/github/RiyaPandita/SMART-VOICE-ASSISTANT-FOR-THE-BLIND-USING-ARTIFICIAL-INTELLIGENCE/blob/main/AIProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Smart Voice Assistant For The Blind**

**Dataset ->** <a href="https://www.kaggle.com/datasets/adityajn105/flickr8k">Flickr 8k Dataset</a>

### **Setting Up Colab and Importing Modules**

In [ ]:
# Mounting google drive into colab
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Setting up the Kaggle Setup
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/MyDrive/kaggle_dataset'

In [ ]:
#Go to the folder
%cd /content/gdrive/MyDrive/kaggle_dataset

In [ ]:
#Setting up the BASE and the WORKING Directories
BASE_DIR = '/content/gdrive/MyDrive/kaggle_dataset'
WORKING_DIR = '/content'

## **Importing the modules and loading the VGG16() model**

In [ ]:
#Importing the necessary modules
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

In [ ]:
# Loading the VGG16
model = VGG16()

#Changing the model: Removing the predicted values from the existing VGG16 model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

## **Extracting features, loading descriptions, and mapping the images with descriptions**

In [ ]:
# Extracting the features into features array
features = {}
directory = os.path.join(BASE_DIR, 'Images')

for i in tqdm(os.listdir(directory)):
    img_path = directory + '/' + i
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    image_id = i.split('.')[0]
    features[image_id] = feature


# Dumping the features array into a pickle file
pickle.dump(features, open(os.path.join(BASE_DIR, 'features.pkl'), 'wb'))

In [ ]:
# load features from the saved pickle file
with open(os.path.join(BASE_DIR, 'features.pkl'), 'rb') as f:
    features = pickle.load(f)

In [ ]:
# Reading the descriptions.txt file
with open(os.path.join(BASE_DIR, 'captions.txt'), 'r') as f:
    next(f)
    desc_doc = f.read()

In [ ]:
#Mapping the descriptions to the images
mapping = {}
for each_desc in tqdm(desc_doc.split('\n')):
    tokens = each_desc.split(',')
    if len(each_desc) < 2:
        continue
    image_id, desc_of = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    desc_of = " ".join(desc_of)
    if image_id not in mapping:
        mapping[image_id] = []
    mapping[image_id].append(desc_of)

## **After mapping, we edit the descriptions: convert them into lower case, remove special characters, etc. and add ‘beginning’ and ‘ending’ tags and later tokenize them i.e., find the unique words.**

In [ ]:
# Editing the descriptions: Convert to lower case and add beginning and ending
def edit_description(mapping):
    for key, desc in mapping.items():
        for i in range(len(desc)):
            x = desc[i]
            x = x.lower()
            x = x.replace('[^A-Za-z]', '')
            x = x.replace('\s+', ' ')
            x = 'beginning ' + " ".join([word for word in x.split() if len(word)>1]) + ' ending'
            desc[i] = x

In [ ]:
# Calling the preprocessing text function
edit_description(mapping)

In [ ]:
# Appending all descriptions into a list: Each image with 5 descriptions
img_desc = []
for key in mapping:
    for caption in mapping[key]:
        img_desc.append(caption)

In [ ]:
# Tokenizing the text: finding the unique words from all the captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(img_desc)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
print("Unique words in the captions are: " + str(vocab_size))

In [ ]:
# Get the maximum description length for the padding required
max_length = max(len(text.split()) for text in img_desc)
max_length

##**Splitting the dataset into training and testing ( 90% into training and 10% into test data)**

In [ ]:
# Splitting the dataset into Training and Testing: 90% is given to training and remaining is for the test
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

In [ ]:
# Generating the data frm the inputs of images and descriptions and passing it for the model
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in data_keys:
            n += 1
            text = mapping[key]
            for t in text:
                seq = tokenizer.texts_to_sequences([t])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield [X1, X2], y
                X1, X2, y = list(), list(), list()
                n = 0

##**Giving the inputs for the CNN.**

In [ ]:
# Giving the inputs for the CNN

inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
from tensorflow.keras.utils import plot_model

# Define the inputs for the model
inputs1 = Input(shape=(4096,))
inputs2 = Input(shape=(max_length,))

# Define the feature extraction layer
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# Define the sequence model layer
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

# Combine the feature extraction and sequence model layers
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# Define the model
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

# Plot the model architecture
plot_model(model, to_file='model.png', show_shapes=True)

##Training the model with 20 epochs

In [ ]:
#Training the model with 20 epochs
epochs = 20
batch_size = 32
steps = len(train) // batch_size

for i in range(epochs):
    generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, batch_size)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

model.save(BASE_DIR+'/best_model.h5')

##**Functions to predict and generate the descriptions for the image**

In [ ]:
def mapping_toword(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [ ]:
def predict_description(model, image, tokenizer, max_length):
    in_text = 'beginning'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], max_length)
        desc_predict = model.predict([image, sequence], verbose=0)

        desc_predict = np.argmax(desc_predict)
        word = mapping_toword(desc_predict, tokenizer)
        if word is None:
            break
        in_text += " " + word
        if word == 'ending':
            break

    return in_text

In [ ]:
actual, predicted = list(), list()

for key in tqdm(test):
    desc = mapping[key]
    y_pred = predict_description(model, features[key], tokenizer, max_length)
    actual_desc = [caption.split() for text in desc]
    y_pred = y_pred.split()
    actual.append(actual_desc)
    predicted.append(y_pred)

In [ ]:
!pip install pyttsx3

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
def generate_text(image_name):
    image_id = image_name.split('.')[0]
    img_path = os.path.join(BASE_DIR, "Images", image_name)
    image = Image.open(img_path)
    desc = mapping[image_id]
    y_pred = predict_description(model, features[image_id], tokenizer, max_length)
    plt.imshow(image)

    return str(y_pred)

In [ ]:
# Installing the required modules

!pip3 install gTTS pyttsx3 playsound pygobject

In [ ]:
pip install SpeechRecognition

## **Function to take the voice input from the user which is the name of the video file which needs to be processed to generate the description.**

In [ ]:
# Setting up the engine for voice to text for input commands
import os
import speech_recognition as sr
import pyttsx3


def voice_output(command):
    engine = pyttsx3.init()
    engine.say(command)
    engine.runAndWait()
r = sr.Recognizer()
x = 0

In [ ]:
!pip install gTTs

In [ ]:
from gtts import gTTS
from IPython.display import Audio

text = str(generate_text("23445819_3a458716c1.jpg"))
print(text)

res = text.split(' ', 1)[1]
text = res.rsplit(' ', 1)[0]

tts = gTTS(text)

tts.save('info.wav')
sound_file = 'info.wav'
Audio(sound_file, autoplay=True)

In [ ]:
text = str(generate_text("1001773457_577c3a7d70.jpg"))
print(text)

res = text.split(' ', 1)[1]
text = res.rsplit(' ', 1)[0]

tts = gTTS(text)

tts.save('info.wav')
sound_file = 'info.wav'
Audio(sound_file, autoplay=True)

In [ ]:
text = str(generate_text("1002674143_1b742ab4b8.jpg"))
print(text)

res = text.split(' ', 1)[1]
text = res.rsplit(' ', 1)[0]

tts = gTTS(text)

tts.save('info.wav')
sound_file = 'info.wav'
Audio(sound_file, autoplay=True)

Time taken for the processes:

1) To extract the features of all the images : 17 min       
2) To train the model (20 epochs): 31 min               
3) To get the description of an image: 2 seconds          
4) To get the description of a video: 21 seconds (Delay identified)